In [1]:
import pyspark
import pandas
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, DateType, ArrayType
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

24/05/31 19:22:11 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/05/31 19:22:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/31 19:22:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/31 19:22:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/31 19:22:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# FUNCIONA
json_fonte = ['{"id_vendedor":1,"valor_faturado":10.0}','{"id_vendedor":2,"valor_faturado":20.0}','{"id_vendedor":2,"valor_faturado":20.0}']

import json
outpur_json = []
for item in json_fonte:
    outpur_json.append(json.loads(item))
print(type(outpur_json))

teste_schema_1 = StructType([
    StructField("id_vendedor", IntegerType(), False),
    StructField("valor_faturado", DoubleType(), False)
])

# teste_data_1 = [(1,10.00),(2,20.00),(2,20.00)]
teste_df_1 = spark.createDataFrame(data=outpur_json,schema=teste_schema_1)
teste_df_1.printSchema()
teste_df_1.show(truncate=False)

<class 'list'>
root
 |-- id_vendedor: integer (nullable = false)
 |-- valor_faturado: double (nullable = false)



+-----------+--------------+
|id_vendedor|valor_faturado|
+-----------+--------------+
|1          |10.0          |
|2          |20.0          |
|2          |20.0          |
+-----------+--------------+



In [20]:
teste_schema_1 = StructType([
    StructField("id_vendedor", IntegerType(), False),
    StructField("valor_faturado", DoubleType(), False)
])
teste_data_1 = [(1,10.00),(2,20.00),(2,20.00)]
teste_df_1 = spark.createDataFrame(data=teste_data_1,schema=teste_schema_1)
teste_df_1.printSchema()
teste_df_1.show(truncate=False)

root
 |-- id_vendedor: integer (nullable = false)
 |-- valor_faturado: double (nullable = false)

+-----------+--------------+
|id_vendedor|valor_faturado|
+-----------+--------------+
|1          |10.0          |
|2          |20.0          |
|2          |20.0          |
+-----------+--------------+



In [21]:
out_teste_df_1 = teste_df_1.select(F.to_json(F.struct(*teste_df_1.columns)).alias("value"))
out_teste_df_1.show(truncate=False)

str_teste_df_1 = out_teste_df_1.select("value").rdd.flatMap(lambda x: x).collect()
str_teste_df_1

isto = ([{"id_vendedor":1,"valor_faturado":10.0},{"id_vendedor":2,"valor_faturado":20.0}])
teste_schema_2 = StructType([
    StructField("faturamento_desconto",teste_schema_1,True)
])

teste_df_1 = spark.createDataFrame(data=isto,schema=teste_schema_2)
teste_df_1.printSchema()
teste_df_1.show(truncate=False)

# saiu_teste_df_1 = str_teste_df_1.select(F.to_json(F.struct(*str_teste_df_1.columns)).alias("saiu"))
# saiu_teste_df_1.show(truncate=False)

+---------------------------------------+
|value                                  |
+---------------------------------------+
|{"id_vendedor":1,"valor_faturado":10.0}|
|{"id_vendedor":2,"valor_faturado":20.0}|
|{"id_vendedor":2,"valor_faturado":20.0}|
+---------------------------------------+

root
 |-- faturamento_desconto: struct (nullable = true)
 |    |-- id_vendedor: integer (nullable = false)
 |    |-- valor_faturado: double (nullable = false)

+--------------------+
|faturamento_desconto|
+--------------------+
|NULL                |
|NULL                |
+--------------------+



EXEMPLOS ABAIXO

In [6]:
jsonString="""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""
df=spark.createDataFrame([(1, jsonString)],["id","value"])
df.show(truncate=False)

+---+--------------------------------------------------------------------------+
|id |value                                                                     |
+---+--------------------------------------------------------------------------+
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+



In [12]:
#Convert JSON string column to Map type
from pyspark.sql.types import MapType,StringType
from pyspark.sql.functions import from_json
df2=df.withColumn("value",from_json(df.value,MapType(StringType(),StringType())))
df2.printSchema()
df2.show(truncate=False)

root
 |-- id: long (nullable = true)
 |-- value: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---+---------------------------------------------------------------------------+
|id |value                                                                      |
+---+---------------------------------------------------------------------------+
|1  |{Zipcode -> 704, ZipCodeType -> STANDARD, City -> PARC PARQUE, State -> PR}|
+---+---------------------------------------------------------------------------+



In [3]:
from pyspark.sql.functions import to_json,col
df2.withColumn("value",to_json(col("value"))) \
   .show(truncate=False)

+---+----------------------------------------------------------------------------+
|id |value                                                                       |
+---+----------------------------------------------------------------------------+
|1  |{"Zipcode":"704","ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+----------------------------------------------------------------------------+



In [4]:
from pyspark.sql.functions import json_tuple
df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City")) \
    .toDF("id","Zipcode","ZipCodeType","City") \
    .show(truncate=False)

+---+-------+-----------+-----------+
|id |Zipcode|ZipCodeType|City       |
+---+-------+-----------+-----------+
|1  |704    |STANDARD   |PARC PARQUE|
+---+-------+-----------+-----------+



In [5]:
from pyspark.sql.functions import get_json_object
df.select(col("id"),get_json_object(col("value"),"$.ZipCodeType").alias("ZipCodeType")) \
    .show(truncate=False)

+---+-----------+
|id |ZipCodeType|
+---+-----------+
|1  |STANDARD   |
+---+-----------+



In [6]:
from pyspark.sql.functions import schema_of_json,lit
schemaStr=spark.range(1) \
    .select(schema_of_json(lit("""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""))) \
    .collect()[0][0]
print(schemaStr)

STRUCT<City: STRING, State: STRING, ZipCodeType: STRING, Zipcode: BIGINT>


In [9]:
from pyspark.sql import SparkSession,Row
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

jsonString="""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""
df=spark.createDataFrame([(1, jsonString)],["id","value"])
df.show(truncate=False)

#Convert JSON string column to Map type
from pyspark.sql.types import MapType,StringType
from pyspark.sql.functions import from_json
df2=df.withColumn("value",from_json(df.value,MapType(StringType(),StringType())))
df2.printSchema()
df2.show(truncate=False)

from pyspark.sql.functions import to_json,col
df2.withColumn("value",to_json(col("value"))) \
   .show(truncate=False)

from pyspark.sql.functions import json_tuple
df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City")) \
    .toDF("id","Zipcode","ZipCodeType","City") \
    .show(truncate=False)

from pyspark.sql.functions import get_json_object
df.select(col("id"),get_json_object(col("value"),"$.ZipCodeType").alias("ZipCodeType")) \
    .show(truncate=False)

from pyspark.sql.functions import schema_of_json,lit
schemaStr=spark.range(1) \
    .select(schema_of_json(lit("""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""))) \
    .collect()[0][0]
print(schemaStr)

+---+--------------------------------------------------------------------------+
|id |value                                                                     |
+---+--------------------------------------------------------------------------+
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+

root
 |-- id: long (nullable = true)
 |-- value: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---+---------------------------------------------------------------------------+
|id |value                                                                      |
+---+---------------------------------------------------------------------------+
|1  |{Zipcode -> 704, ZipCodeType -> STANDARD, City -> PARC PARQUE, State -> PR}|
+---+---------------------------------------------------------------------------+

+---+---------------------------------